In [ ]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaTableInst") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040920 sha256=94985a80dca104b1e0f42644f13e43255f24fd835825d8d4e8f4a2ad084ef62a
  Stored in directory: /root/.cache/pip/wheels/81/6c/a6/af459ed3106da8f1171a265c2982425fe390aa418a47d2a8b4
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
df=spark.read.format('csv').option('header','true').option('inferSchema',True).load('/content/drive/MyDrive/Abc/Ola_data/products.csv')

In [ ]:
df.write.format('delta').mode('overwrite').saveAsTable('Products_table')

In [ ]:
deltaInstance1=DeltaTable.forPath(spark,'/content/spark-warehouse/products_table')

In [ ]:
deltaInstance1.toDF().show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



In [ ]:
spark.sql("delete from Products_table where product_id='PROD101'")

DataFrame[]

In [ ]:
spark.sql('select * from Products_table').show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



In [ ]:
deltaInstance1.toDF().show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



both the deltainstance and table are pointing to same memory instance, hence we could see replica as same when we query using SQL and DeltaInstance

Using forName table Name

In [ ]:
deltaInstance2=DeltaTable.forName(spark,'Products_table')

In [ ]:
deltaInstance2.toDF().show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



As can be seen, using forName also pointing to same deltatable that been created above

#### Describing History

In [ ]:
spark.sql('Describe History Products_table').show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+--------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                             |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+-

In [ ]:
deltaInstance1.history().show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+--------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                             |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+-

In [ ]:
df.write.format('delta').mode('append').saveAsTable('products_table')

In [ ]:
spark.sql('describe History products_table').show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2025-07-21 07:02:...|  null|    null|               WRITE|{mode -> Append, ...|null|    null|     null|          1|  Serializable|         true|{numFiles -> 1, n...|        null|Apache-Spark/3.2....|
|      1|2025-07-21 07:02:...|  null|    null|              DELETE|{predicate -> ["(...|null|    null|     null|          0|  Serializable|        false|{numRemoved

In [ ]:
df.createOrReplaceTempView('temp_table')


In [ ]:
spark.sql('select * from temp_table').show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



####Delete data from Delta tables

Using deltatable location

In [ ]:
spark.sql("delete from delta.`/content/spark-warehouse/products_table` where product_id like('PROD101')")
spark.sql('select * from delta.`/content/spark-warehouse/products_table`').show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



In [ ]:
spark.sql('describe history delta.`/content/spark-warehouse/products_table`').show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+-

In [ ]:
deltaInstance3=DeltaTable.forPath(spark,'/content/spark-warehouse/products_table')

In [ ]:
deltaInstance3.delete('product_id like("PROD103") and country="USA"')

In [ ]:
deltaInstance3.toDF().show(5)

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
+-------------------+---------------+----------+--------+------+----------+-------+
only showing top 5 rows



In [ ]:
deltaInstance3.history().show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+-

In [ ]:
spark.sql('update products_table set store_id="STORE03" where product_id="PROD102"')
spark.sql('describe history products_table').show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+-

In [ ]:
deltaInstance3.update('product_id="PROD104"',set={"store_id":lit("STORE03")})

In [ ]:
deltaInstance3.history().show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+-

#TIME TRAVEL

There are two ways to timetravel, in spark,
1. Using timestampAsOf
2.Using versionAsOf

In [ ]:
df=spark.read.format('delta').option("timestampAsOf","2025-07-21 07:20:30.024").table('products_table')
df.show(5)

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
+-------------------+---------------+----------+--------+------+----------+-------+
only showing top 5 rows



In [ ]:
df1=spark.read.format('delta').option("versionAsOf",3).table('products_table')
df1.show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



Using SQL

In [ ]:
#spark.sql('select * from products_table3 version as of 3').show()
#spark.sql("select * from products_table3 timestamp as of '2025-07-21 03:00:18.113';")
#spark.sql("select * from delta.'/content/spark-warehouse/products_table3' VERSION AS OF 3;")

### Restore

A restore command make the table to permanently to previous version, where as in Time travel, we only travel back and forth without changing the current state.

In [ ]:
spark.sql('describe history delta.`/content/spark-warehouse/products_table`').show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+-

In [ ]:
product_instance=DeltaTable.forPath(spark,'/content/spark-warehouse/products_table')

In [ ]:
product_instance.history().show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                  |userMetadata|engineInfo                         |
+-------+-----------------------+-

In order to restore to previous version, then we can use
1. restoreToVersion('VersionNumber')
2. restoreToVersion('Timestamp')

In [ ]:
product_instance.restoreToVersion(4)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [ ]:
product_instance.toDF().show()

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
|             Webcam|Electronics Hub|   PROD108| STORE01| 55.00|      NULL|    USA|
|     Gaming Headset|      PC Palace|   PROD109| STORE04| 90.00|  New York|    USA|
|      Monitor Stand|   Home Gadgets|   PROD110| STORE03| 29.99|   Florida|    USA|
|         HDMI Cable|    Mobile Mart|   PROD111| STORE02|  NULL|     Texas| 

In [ ]:
product_instance.restoreToTimestamp('2025-07-21 07:19:56.058')

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [ ]:
product_instance.toDF().show()

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|  Bluetooth Speaker|Electronics Hub|   PROD103| STORE01| 79.50|California|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
|             Webcam|Electronics Hub|   PROD108| STORE01| 55.00|      NULL|    USA|
|     Gaming Headset|      PC Palace|   PROD109| STORE04| 90.00|  New York|    USA|
|      Monitor Stand|   Home Gadgets|   PROD110| STORE03| 29.99|   Florida| 

In [ ]:
product_instance.history().show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      8|2025-07-21 07:41:...|  null|    null|             RESTORE|{version -> null,...|null|    null|     null|          7|  Serializable|        false|{numRestoredFiles...|        null|Apache-Spark/3.2....|
|      7|2025-07-21 07:38:...|  null|    null|             RESTORE|{version -> 4, ti...|null|    null|     null|          6|  Serializable|        false|{numRestore

**END OF CODE**